<a href="https://colab.research.google.com/github/youngpaper1115/Git_Tutorial/blob/main/Lending_Club_Project_ver1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# google drive mount 및 파일 경로 지정
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# 필요 라이브러리 import 및 현재 작업공간 확인
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

%matplotlib notebook
%matplotlib inline

os.getcwd()

'/content'

In [25]:
# 데이터 불러오기

lc_df = pd.read_stata('/content/drive/MyDrive/Python_SNU/Lending_Club/lendingclub.dta')

# **<< Column에 대한 생각 정리>>**

1. **loan_amnt**, funded_amnt, funded amnt_inv : 대부분 같은 값으로 loan_amnt 컬럼으로 통일

2. **int_rate** : 이자율.. 유지

3. installment : 대출자의 월 지급액. 총량 / 기간으로 계산되므로 del

4. **annual_inc** : 대출자의 자진신고 연소득.. 유지

5. **dti** : ??? 이해 안감...ㅜㅜ 설명좀.. --> 대출 실질 상환 가능성?? 넣는 것으로. 높으면 위험한 값

6. **delinq_2yrs** : 근 2년간 대출자의 연체 발생일수 30일 이상.. 유지

7. fico_range_low, fico_range_high : 대출자의 FICO 하한&상한 범위? FICO란? 포함시킬 경우, 둘의 평균값 넣는 것이 나을 듯

8. **inq_last_6mths** : 6개월간 조회수.

9. **open_acc** : 신용파일이 열린 수. / 카드 여러개..

10. **pub_rec** : 나쁜 공공기록의 수..? 신용등급이 낮은 경우일 것 같으므로 우선 포함

11. revol_bal, **revol_util** : 리볼빙(광역할부..라 합니다) 금액 및 소비율. 소비율이 낮을수록 연체 비율이 줄지 않을까..? 우선 소비율만 포함

12. **total_acc** : 대출자 신용 파일의 총 신용 한도 수.. 데이터 확인 (open_acc와 비교... total은 현재 전체 계좌 보유 수, open이 계좌 개설 수)

13. out_prncp, out_prncp_inv : 잔존 원금들.. 연체면 0이 아닐 것.. 우선 제외 -> 제외 이유는 말할 것. 체크 필요

14. total_pymnt, total_pymnt_inv : 총 지급액.. 원금*이자율 결과물일테니 제외

15. total_rec_prncp, total_rec_int, total_rec_late_fee : 수령한 원금, 이자, 연체료. 원금 및 이자는 불필요. 연체료는 depvar이 0인 경우에 발생하기도 하고, 1인 경우에 발생하지 않은 경우도 있어서.. 제외. late_fee의 경우는 위의 delinq_2yrs 컬럼과 상관성 확인 필요

16. recoveries, **collection_recovery_fee** : 총회복 연기, 추심료 우송..? 개념은 이해 안감. 단, 종속변수가 1인 경우에만 발생(물론 0인 경우도 존재). recoveries를 5.555..로 나눈 값이 collection_recovery_fee 인듯. 뒤 값만 사용 -> 개인 회생 관련 목록이 있으면 제외

17. last_pymnt_amnt : 마지막 받은 총 지급액.. 제외

18. last_fico_range_high, last_fico_range_low : FICO가 대체 뭐람.. 포함시킬 경우 둘의 평균값을 넣자

19. **collections_12_mths_ex_med** : 의료수거 제외 12개월간 수거 횟수..? -> 의료비로 인한 상환 불가 경우 제외하고 채권 회수 건수. Classification에 쓸 수 있을 듯

20. **acc_now_delinq** : 현재 연체 중인 계정 수.. 우선 포함 (loan_amnt 대비율 살펴보기)

21. tot_coll_amt, **tot_cur_bal** : 총 추심 금액 및 모든 계정의 경상 잔액. (cur는 모든 금융권의 남은 대출 잔액. 타금융권의 계좌도 확인 필요)

22. **chargeoff_within_12_mths** : 손실 처리 횟수.

23. **delinq_amnt** : 현재 연체 중인 계좌의 연체 금액. 위에 연체 계정 수가 있으니 우선 제외 -> 금액은 계좌 수와 다른 것이니 추가

24. **pub_rec_bankruptcies** : 공공 파산 기록 건수. -> 상관계수 보고 결정..

25. **tax_liens** : 세금 안내서 압류잡힌거..ㅜ -> 세금이 우선순위이기에 이것이 있으면 상환 못할 확률 높음

26. ***emp_length*** 1 ~ 12 : 고용 기간. 설명에는 0~10까지의 값만을 가진다고 하였지만 column은 12까지 존재. 이 column은 emp_length 하나의 column으로 통일한 후, 각 가능 고용기간을 int로 넣겠음

27. home_ownership 1~6 : 주택 보유 현황.. 이건 종속 변수와의 관계를 봐야할 것 같다. -> 관계 확인 필요

28. verification_status 1~3 : 소득의 확인 여부... 대출자의 신뢰도를 판단하는 기준이 될 수 있을지도? -> 관계 확인 필요

29. purpose 1~14 : 제외

30. addr_state 1~51 : 대출자가 살고 있는 주. 우선 제외

31. **initial_list_status** : 대출 초기 목록 상태(W : 가조회, F : 본조회) -> 조교님께 물어보기 : 한명한테 빌리냐(W) or 여럿한테 빌리냐(F)

32. elapsed_t : 뭐야 넌 --> 작년엔 사용하지 않은 변수

33. **issue_d** 1~118 : 융자금 지원 달.. 우선 int화 
시키기 --> 융자금을 받은 후 지난 일수.. issue_d1의 경우는 579+1일  -> 조교님/께 문의

34. **debt_settlement_flag** : 채무 조정 회사와 협력 여부. 0인 경우 종속변수가 1되는 경우가 몇몇 있는 듯

35. **term** : 지불 수. 월 단위로 36 or 60  --> 조교님께 문의

36. **depvar : 종속변수.**

37. **mths_since_last_delinq** 1~11 : 대출자의 마지막 연체 이후 개월 수. 이것도 우선 int화 -> 조교님게 여쭤보기. Delinquent_1인 경우가 바로 저번달 연체 경우가 맞는지?

38. **mths_since_last_major_derog** 1~11: 해석이 잘.. 우선 int화 -> 조교님께 여쭤보기. 90일 또는 그 이상 연체 후 지난 달이 맞는가? 또한 연체가 없었을 경우 derog1으로 처리???

39. **mths_since_last_record** 1~11 : 마지막 공공 기록 이후 개월 수? int화

40. **mths_since_rcnt_il** 1~11 : 최근 할부 계좌 open 후 월 int화. -> 조교님께 문의..

41. **mths_since_recent_bc** 1 ~ 11 : 은행카드 개설 후 지난 월 수. int화

42. **mths_since_recent_bc_dlq** 1 ~ 11 : 은행카드 연체 이후 월 수. int화

43. **mths_since_recent_inq** 1 ~ 10 : 최근 문의 이후 월 수. int화

44. mths_since_recent_revol_delinq 1 ~ 11 : 최근 리볼빙 불량 이후 지난 월 수. int화


==> 마지막 변수들 month가 정말 달을 말하는 것인가 or구간을 말하는 것인가? 0이나 11인 경우는 이하or이상의 값까지 볼 것인가

### <컬럼 분석 시작>


*   columns1, columns2... 형식으로 처리된 변수 분석 함수 정의





In [26]:
def make_pivot_table(columns, lc_df):
  # 특정변수와 종속변수만 불러온 임시 데이터프레임 생성
  temp_list = list(filter(lambda x: columns in x, lc_df.columns.tolist())) # 입력 컬럼명만 포함되는 컬럼리스트 추출
  if columns == 'mths_since_recent_bc':
    temp_list = temp_list[:11]
  temp_list.append('depvar') # 변환 리스트에 종속변수 'depvar' 값 추가
  temp_df = lc_df[temp_list]

  # 우량 / 불량 집단으로 분류
  good_temp = temp_df[temp_df['depvar']==0] # depvar=0의 경우, 상환완료
  bad_temp = temp_df[temp_df['depvar']==1] # depvar=1의 경우, 상환x

  # 우량 / 불량 집단 별 합계
  good_sum = good_temp.sum(axis=0)
  bad_sum = bad_temp.sum(axis=0)

  # 전체 값 대비 비율 생성
  temp_ratio = []
  for i in temp_list[:-1]:
    temp_ratio.append(len(temp_df[temp_df[i]==1]) / len(temp_df))

  # index 별 전체 대비 불량 비율(불량률)
  bad_ratio = round((bad_sum[:-1] / (good_sum[:-1] + bad_sum[:-1]))*100,2)

  pivot_table = pd.DataFrame(index=temp_list[:-1])

  temp_ratio=[]
  for i in temp_list[:-1]:
    value = len(temp_df[temp_df[i]==1]) / len(temp_df)
    temp_ratio.append(str(round(value*100, 2)) + '%')
  
  pivot_table['whole(%)'] = temp_ratio
  pivot_table['good'] = good_sum[:-1]
  pivot_table['bad'] = bad_sum[:-1]
  pivot_table['bad ratio(%)'] = bad_ratio

  return pivot_table

1. emp_length 1~12

In [27]:
a = make_pivot_table('emp_length', lc_df)
a
# 고용 기간이 김에도 불구하고 빌리는 사람들은 그만큼 궁할거다. 대출수요가 높다. 치명적 결함이다.
# emp_length12만 기입하여 0 or 1로 구분

,whole(%),good,bad,bad ratio(%)
emp_length1,6.54%,59573.0,11852.0,16.59
emp_length2,33.02%,307028.0,53877.0,14.93
emp_length3,9.02%,82747.0,15832.0,16.06
emp_length4,7.99%,73212.0,14087.0,16.14
emp_length5,5.82%,53418.0,10235.0,16.08
emp_length6,6.14%,56296.0,10765.0,16.05
emp_length7,4.66%,42634.0,8257.0,16.22
emp_length8,4.48%,40931.0,8051.0,16.44
emp_length9,4.66%,42621.0,8256.0,16.23
emp_length10,3.86%,35337.0,6845.0,16.23


2. home_ownership 1~6

In [28]:
a = make_pivot_table('home_ownership', lc_df)
a
# 1,2 / 3,4,5,6

,whole(%),good,bad,bad ratio(%)
home_ownership1,0.03%,331.0,51.0,13.35
home_ownership2,48.11%,453146.0,72688.0,13.82
home_ownership3,0.0%,35.0,7.0,16.67
home_ownership4,0.0%,37.0,8.0,17.78
home_ownership5,10.94%,99788.0,19789.0,16.55
home_ownership6,40.9%,362758.0,84281.0,18.85


3. mths_since_last_delinq 1~11

In [29]:
a = make_pivot_table('mths_since_last_delinq', lc_df)
a
# 1 / 2~11

,whole(%),good,bad,bad ratio(%)
mths_since_last_delinq1,49.09%,451988.0,84520.0,15.75
mths_since_last_delinq2,5.22%,47379.0,9669.0,16.95
mths_since_last_delinq3,5.87%,53179.0,11025.0,17.17
mths_since_last_delinq4,4.57%,41494.0,8431.0,16.89
mths_since_last_delinq5,5.1%,46674.0,9077.0,16.28
mths_since_last_delinq6,4.84%,44067.0,8824.0,16.68
mths_since_last_delinq7,5.1%,46702.0,9073.0,16.27
mths_since_last_delinq8,5.31%,48689.0,9298.0,16.03
mths_since_last_delinq9,4.95%,45253.0,8817.0,16.31
mths_since_last_delinq10,4.94%,44970.0,9011.0,16.69


4. mths_since_last_major_derog

In [30]:
a = make_pivot_table('mths_since_last_major_derog', lc_df)
a
# 1 or 2~11로 구분

,whole(%),good,bad,bad ratio(%)
mths_since_last_major_derog1,72.14%,665237.0,123212.0,15.63
mths_since_last_major_derog2,2.87%,25600.0,5714.0,18.25
mths_since_last_major_derog3,3.04%,27392.0,5872.0,17.65
mths_since_last_major_derog4,2.78%,25058.0,5275.0,17.39
mths_since_last_major_derog5,2.86%,25693.0,5555.0,17.78
mths_since_last_major_derog6,2.58%,23365.0,4827.0,17.12
mths_since_last_major_derog7,2.95%,26699.0,5488.0,17.05
mths_since_last_major_derog8,2.64%,23801.0,5052.0,17.51
mths_since_last_major_derog9,2.59%,23309.0,4966.0,17.56
mths_since_last_major_derog10,2.88%,25933.0,5515.0,17.54


5. mths_since_last_record

In [31]:
a = make_pivot_table('mths_since_last_record', lc_df)
a
# 1 or 2~11로 구분

,whole(%),good,bad,bad ratio(%)
mths_since_last_record1,82.59%,761022.0,141638.0,15.69
mths_since_last_record2,1.78%,15947.0,3456.0,17.81
mths_since_last_record3,1.76%,15897.0,3319.0,17.27
mths_since_last_record4,1.74%,15769.0,3298.0,17.30
mths_since_last_record5,1.88%,16905.0,3682.0,17.89
mths_since_last_record6,1.78%,15986.0,3507.0,17.99
mths_since_last_record7,1.8%,15976.0,3643.0,18.57
mths_since_last_record8,1.65%,14476.0,3504.0,19.49
mths_since_last_record9,1.66%,14498.0,3635.0,20.05
mths_since_last_record10,1.78%,15566.0,3885.0,19.97


6. mths_since_rcnt_il

In [32]:
a = make_pivot_table('mths_since_rcnt_il', lc_df)
a
# 1 or 2~5, 6~11로 구분

,whole(%),good,bad,bad ratio(%)
mths_since_rcnt_il1,64.4%,587564.0,116243.0,16.52
mths_since_rcnt_il2,5.15%,45127.0,11181.0,19.86
mths_since_rcnt_il3,3.14%,28259.0,6077.0,17.70
mths_since_rcnt_il4,3.23%,29196.0,6146.0,17.39
mths_since_rcnt_il5,2.76%,25013.0,5103.0,16.94
mths_since_rcnt_il6,4.03%,37296.0,6780.0,15.38
mths_since_rcnt_il7,3.24%,30263.0,5158.0,14.56
mths_since_rcnt_il8,3.65%,34325.0,5566.0,13.95
mths_since_rcnt_il9,3.36%,31896.0,4791.0,13.06
mths_since_rcnt_il10,3.48%,33313.0,4736.0,12.45


7. mths_since_recent_bc

In [33]:
a = make_pivot_table('mths_since_recent_bc', lc_df)
a
# 1~6 / 7~11로 구분

,whole(%),good,bad,bad ratio(%)
mths_since_recent_bc1,0.92%,8301.0,1708.0,17.06
mths_since_recent_bc2,13.56%,119228.0,29018.0,19.57
mths_since_recent_bc3,9.09%,81443.0,17925.0,18.04
mths_since_recent_bc4,8.0%,71866.0,15529.0,17.77
mths_since_recent_bc5,10.08%,91317.0,18903.0,17.15
mths_since_recent_bc6,11.19%,101439.0,20911.0,17.09
mths_since_recent_bc7,8.37%,76561.0,14918.0,16.31
mths_since_recent_bc8,10.24%,94818.0,17101.0,15.28
mths_since_recent_bc9,9.42%,88058.0,14885.0,14.46
mths_since_recent_bc10,9.28%,87858.0,13542.0,13.36


8. mths_since_recent_bc_dlq

In [34]:
a = make_pivot_table('mths_since_recent_bc_dlq', lc_df)
a
# 1 / 2~11

,whole(%),good,bad,bad ratio(%)
mths_since_recent_bc_dlq1,74.48%,684097.0,129915.0,15.96
mths_since_recent_bc_dlq2,2.71%,24523.0,5079.0,17.16
mths_since_recent_bc_dlq3,2.61%,23558.0,4988.0,17.47
mths_since_recent_bc_dlq4,2.61%,23726.0,4754.0,16.69
mths_since_recent_bc_dlq5,2.62%,23835.0,4790.0,16.73
mths_since_recent_bc_dlq6,2.57%,23596.0,4485.0,15.97
mths_since_recent_bc_dlq7,2.54%,23294.0,4511.0,16.22
mths_since_recent_bc_dlq8,2.35%,21492.0,4215.0,16.40
mths_since_recent_bc_dlq9,2.56%,23149.0,4824.0,17.25
mths_since_recent_bc_dlq10,2.48%,22472.0,4663.0,17.18


9. mths_since_recent_inq

In [35]:
a = make_pivot_table('mths_since_recent_inq', lc_df)
a
# 1 / 2~6 / 7~11

,whole(%),good,bad,bad ratio(%)
mths_since_recent_inq1,10.63%,103055.0,13166.0,11.33
mths_since_recent_inq2,17.91%,156642.0,39048.0,19.95
mths_since_recent_inq3,15.25%,136286.0,30420.0,18.25
mths_since_recent_inq4,6.59%,59622.0,12388.0,17.20
mths_since_recent_inq5,5.87%,53451.0,10676.0,16.65
mths_since_recent_inq6,10.23%,93687.0,18149.0,16.23
mths_since_recent_inq7,8.21%,75827.0,13932.0,15.52
mths_since_recent_inq8,9.05%,84106.0,14757.0,14.93
mths_since_recent_inq9,8.05%,75389.0,12578.0,14.30
mths_since_recent_inq10,8.21%,78030.0,11710.0,13.05


10. mths_since_recent_revol_delinq

In [36]:
a = make_pivot_table('mths_since_recent_revol_delinq', lc_df)
a
# out

,whole(%),good,bad,bad ratio(%)
mths_since_recent_revol_delinq1,64.49%,592017.0,112845.0,16.01
mths_since_recent_revol_delinq2,3.84%,34923.0,7024.0,16.74
mths_since_recent_revol_delinq3,3.81%,34489.0,7100.0,17.07
mths_since_recent_revol_delinq4,3.61%,32937.0,6521.0,16.53
mths_since_recent_revol_delinq5,3.45%,31467.0,6277.0,16.63
mths_since_recent_revol_delinq6,3.23%,29449.0,5826.0,16.52
mths_since_recent_revol_delinq7,3.58%,32962.0,6118.0,15.66
mths_since_recent_revol_delinq8,3.78%,34690.0,6613.0,16.01
mths_since_recent_revol_delinq9,3.36%,30727.0,6005.0,16.35
mths_since_recent_revol_delinq10,3.58%,32543.0,6550.0,16.75


In [37]:
# mths_1 의 경우에는 o/x의 경우로 먼저 분류한 후 segment한 것이라고 가정
# mths_2~11의 경우에는 서열효과가 있다고 판단되는 경우네는 다시 이들을 나눠서 분석 진행
# mths_2~11의 경우에는 서열효과가 있다고 판단되는 경우 다른 분석법이 있는지 한번 찾아보고 공유

<내가 원했던 컬럼>

In [38]:
jong0 = ['installment', 'annual_inc', 'delinq_2yrs', 'fico_range_low', 'inq_last_6mths', 'out_prncp', 'out_prncp_inv', 'last_pymnt_amnt', 'last_fico_range_low', 'collections_12_mths_ex_med', 
         'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens']

In [39]:
# 컬럼별 고유값 개수

for col in jong0:
  print(col + ' : ' + str(len(lc_df[col].value_counts())))

installment : 70654
annual_inc : 55905
delinq_2yrs : 26
fico_range_low : 38
inq_last_6mths : 9
out_prncp : 180
out_prncp_inv : 180
last_pymnt_amnt : 518735
last_fico_range_low : 71
collections_12_mths_ex_med : 16
acc_now_delinq : 8
tot_coll_amt : 12270
tot_cur_bal : 366324
chargeoff_within_12_mths : 10
delinq_amnt : 1887
pub_rec_bankruptcies : 12
tax_liens : 37


1. 고유값 개수가 적은 경우

In [40]:
def make_pivot_table2(columns, lc_df):
  # 컬럼명과 종속변수만 추출
  temp_df = lc_df[[columns,'depvar']]

  # 우량/불량 집단으로 분류
  good_temp = temp_df[temp_df['depvar']==0] # depvar=0의 경우, 상환완료
  bad_temp = temp_df[temp_df['depvar']==1] # depvar=1의 경우, 상환x

  # 피벗테이블 생성
  pivot_table = pd.DataFrame(index=temp_df[columns].value_counts().index)

  temp_ratio = []
  for i in temp_df[columns].value_counts().index.tolist():
    temp_ratio.append(str(round((temp_df[columns].value_counts()[i] / len(temp_df))*100,2))+'%')

  pivot_table['whole(%)'] = temp_ratio
  pivot_table['good'] = good_temp[columns].value_counts()
  pivot_table['bad'] = bad_temp[columns].value_counts()
  pivot_table['bad ratio(%)'] = round((pivot_table['bad'] / (pivot_table['good'] + pivot_table['bad']))*100,2)

  return pivot_table
  

In [41]:
# delinq_2yrs

a = make_pivot_table2('delinq_2yrs', lc_df)
a

,whole(%),good,bad,bad ratio(%)
0,79.64%,731272,139099.0,15.98
1,13.38%,122089,24141.0,16.51
2,4.02%,36259,7629.0,17.38
3,1.48%,13167,2985.0,18.48
4,0.67%,6027,1315.0,17.91
5,0.34%,3031,681.0,18.35
6,0.19%,1705,377.0,18.11
7,0.11%,925,230.0,19.91
8,0.06%,559,114.0,16.94
9,0.04%,340,82.0,19.43


In [42]:
# fico_range_low

a = make_pivot_table2('fico_range_low', lc_df)
a

,whole(%),good,bad,bad ratio(%)
660,9.52%,81338,22749,21.86
665,9.17%,79415,20790,20.75
670,9.12%,79869,19847,19.90
675,8.07%,71227,17023,19.29
680,7.91%,70298,16110,18.64
685,6.9%,62322,13062,17.33
690,6.69%,60926,12179,16.66
695,5.96%,54840,10285,15.79
700,5.4%,50325,8715,14.76
705,4.87%,45781,7404,13.92


In [43]:
# inq_last_6mths

a = make_pivot_table2('inq_last_6mths', lc_df)
a

,whole(%),good,bad,bad ratio(%)
0,58.17%,545744,89995.0,14.16
1,26.89%,242111,51730.0,17.60
2,9.79%,85044,21944.0,20.51
3,3.61%,30386,9042.0,22.93
4,1.09%,9027,2891.0,24.26
5,0.38%,3119,1016.0,24.57
6,0.08%,660,206.0,23.79
7,0.0%,3,NaN,NaN
8,0.0%,1,NaN,NaN


In [44]:
# collections_12_mths_ex_med

a = make_pivot_table2('collections_12_mths_ex_med', lc_df)
a

,whole(%),good,bad,bad ratio(%)
0,98.37%,901764,173297.0,16.12
1,1.51%,13295,3256.0,19.67
2,0.1%,863,237.0,21.55
3,0.01%,107,22.0,17.05
4,0.0%,34,9.0,20.93
5,0.0%,15,1.0,6.25
6,0.0%,4,2.0,33.33
7,0.0%,3,NaN,NaN
20,0.0%,2,NaN,NaN
10,0.0%,2,NaN,NaN


In [45]:
# acc_now_delinq

a = make_pivot_table2('acc_now_delinq', lc_df)
a

,whole(%),good,bad,bad ratio(%)
0,99.45%,911142.0,175798.0,16.17
1,0.51%,4659.0,958.0,17.06
2,0.03%,251.0,56.0,18.24
3,0.0%,31.0,8.0,20.51
4,0.0%,9.0,2.0,18.18
5,0.0%,2.0,1.0,33.33
6,0.0%,NaN,1.0,NaN
14,0.0%,1.0,NaN,NaN


In [46]:
# chargeoff_within_12_mths

a = make_pivot_table2('chargeoff_within_12_mths', lc_df)
a

,whole(%),good,bad,bad ratio(%)
0,99.15%,908431,175242.0,16.17
1,0.77%,6961,1455.0,17.29
2,0.06%,572,96.0,14.37
3,0.01%,83,17.0,17.00
4,0.0%,25,9.0,26.47
5,0.0%,12,3.0,20.00
6,0.0%,4,2.0,33.33
7,0.0%,3,NaN,NaN
9,0.0%,3,NaN,NaN
10,0.0%,1,NaN,NaN


In [47]:
# pub_rec_bankruptcies

a = make_pivot_table2('pub_rec_bankruptcies', lc_df)
a

,whole(%),good,bad,bad ratio(%)
0,87.8%,807758.0,151798.0,15.82
1,11.38%,101176.0,23219.0,18.67
2,0.63%,5503.0,1384.0,20.10
3,0.14%,1211.0,287.0,19.16
4,0.03%,292.0,81.0,21.72
5,0.01%,109.0,31.0,22.14
6,0.0%,28.0,14.0,33.33
7,0.0%,12.0,5.0,29.41
8,0.0%,4.0,4.0,50.00
9,0.0%,1.0,NaN,NaN


In [48]:
# tax_liens

a = make_pivot_table2('tax_liens', lc_df)
a

,whole(%),good,bad,bad ratio(%)
0,96.29%,882796.0,169624.0,16.12
1,2.5%,22421.0,4882.0,17.88
2,0.69%,6229.0,1359.0,17.91
3,0.26%,2310.0,480.0,17.20
4,0.12%,1034.0,226.0,17.94
5,0.06%,573.0,111.0,16.23
6,0.03%,310.0,56.0,15.30
7,0.02%,149.0,31.0,17.22
8,0.01%,83.0,10.0,10.75
9,0.01%,58.0,7.0,10.77


2. 고유값 개수가 많은 경우: 범주화하여 정리 -> 이번에 만들어진 함수에는 n(갖고 싶은 범주 개수)를 입력할 수 있도록 수정

In [51]:
from pandas.core.reshape.pivot import pivot_table
def make_pivot_table3(columns,n,lc_df):
  # 컬럼명과 종속변수만 추출
  temp_df = lc_df[[columns, 'depvar']]

  # 구간 나누기 및 class 열 생성
  temp_df['class'] = pd.cut(temp_df[columns], n, labels=list(range(n)))

  # 우량 / 불량 집단으로 분류
  good_temp = temp_df[temp_df['depvar']==0] # depvar=0의 경우, 상환완료
  bad_temp = temp_df[temp_df['depvar']==1] # depvar=1의 경우, 상환x

  # 피벗테이블 생성
  pivot_table = pd.DataFrame(index=temp_df['class'].value_counts().index.tolist())

  temp_ratio=[]
  for i in temp_df['class'].value_counts().index.tolist():
    temp_ratio.append(round((len(temp_df[temp_df['class']==i]) / len(temp_df))*100,2))

  pivot_table['whole(%)'] = temp_ratio
  pivot_table['good'] = good_temp['class'].value_counts()
  pivot_table['bad'] = bad_temp['class'].value_counts()
  pivot_table['bad ratio(%)'] = round((pivot_table['bad'] / (pivot_table['good'] + pivot_table['bad']))*100,2)

  pivot_table = pivot_table.sort_index(ascending=True)
  
  return pivot_table

In [52]:
# installment
a = make_pivot_table3('installment',10,lc_df)
a

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,whole(%),good,bad,bad ratio(%)
0,14.72,138653,22183,13.79
1,32.95,302956,57130,15.87
2,22.62,205541,41703,16.87
3,14.14,127630,26889,17.40
4,7.54,69052,13392,16.24
5,3.98,35958,7524,17.30
6,2.72,25048,4706,15.82
7,1.26,10705,3051,22.18
8,0.07,547,243,30.76
9,0.00,5,3,37.50


In [53]:
# annual_inc
a = make_pivot_table3('annual_inc',10,lc_df)
a

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,whole(%),good,bad,bad ratio(%)
0,100.0,916076,176820,16.18
1,0.0,18,4,18.18
2,0.0,0,0,NaN
3,0.0,0,0,NaN
4,0.0,0,0,NaN
5,0.0,0,0,NaN
6,0.0,0,0,NaN
7,0.0,0,0,NaN
8,0.0,0,0,NaN
9,0.0,1,0,0.00


In [54]:
# 위 annual_inc 컬럼에서 너무 큰 이상치가 있어 이 값을 제외하고 다시 실행
temp_index = lc_df[lc_df['annual_inc']==lc_df['annual_inc'].max()].index
temp_df = lc_df.drop(temp_index)

# annual_inc
a = make_pivot_table3('annual_inc',10,temp_df)
a

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,whole(%),good,bad,bad ratio(%)
0,99.97,915846,176790,16.18
1,0.02,187,19,9.22
2,0.00,16,1,5.88
3,0.00,9,3,25.00
4,0.00,8,3,27.27
5,0.00,7,4,36.36
6,0.00,5,0,0.00
7,0.00,9,1,10.00
8,0.00,5,0,0.00
9,0.00,2,3,60.00


In [55]:
# out_prncp
a = make_pivot_table3('out_prncp',7,temp_df)
a

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,whole(%),good,bad,bad ratio(%)
0,99.99,916094,176742,16.17
1,0.00,0,53,100.00
2,0.00,0,16,100.00
3,0.00,0,5,100.00
4,0.00,0,4,100.00
5,0.00,0,3,100.00
6,0.00,0,1,100.00


In [56]:
# out_prncp_inv
a = make_pivot_table3('out_prncp_inv',7,temp_df)
a

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,whole(%),good,bad,bad ratio(%)
0,99.99,916094,176742,16.17
1,0.00,0,53,100.00
2,0.00,0,16,100.00
3,0.00,0,5,100.00
4,0.00,0,4,100.00
5,0.00,0,3,100.00
6,0.00,0,1,100.00


In [57]:
# last_pymnt_amnt
a = make_pivot_table3('last_pymnt_amnt',10,temp_df)
a

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,whole(%),good,bad,bad ratio(%)
0,69.18,579499,176631,23.36
1,14.23,155408,115,0.07
2,7.60,83067,32,0.04
3,4.23,46218,14,0.03
4,2.33,25496,13,0.05
5,1.21,13205,10,0.08
6,0.66,7199,5,0.07
7,0.37,4046,2,0.05
8,0.17,1822,2,0.11
9,0.01,134,0,0.00


In [58]:
# last_fico_range_low
a = make_pivot_table3('last_fico_range_low',10,temp_df)
a

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,whole(%),good,bad,bad ratio(%)
0,2.40,6040,20167,76.95
1,0.00,0,0,NaN
2,0.00,0,0,NaN
3,0.00,0,0,NaN
4,0.00,0,0,NaN
5,1.09,3245,8677,72.78
6,14.85,69230,93022,57.33
7,25.61,230545,49359,17.63
8,42.65,460974,5200,1.12
9,13.40,146060,399,0.27


In [59]:
# tot_coll_amt
a = make_pivot_table3('tot_coll_amt',10,temp_df)
a

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,whole(%),good,bad,bad ratio(%)
0,100.0,916092,176824,16.18
1,0.0,1,0,0.00
2,0.0,0,0,NaN
3,0.0,0,0,NaN
4,0.0,0,0,NaN
5,0.0,0,0,NaN
6,0.0,0,0,NaN
7,0.0,0,0,NaN
8,0.0,0,0,NaN
9,0.0,1,0,0.00


In [60]:
# 위 tot_call_amt 컬럼에서 너무 큰 이상치가 5개 있어 이 값을 제외하고 다시 실행

temp_index = lc_df[lc_df['tot_coll_amt']==lc_df['tot_coll_amt'].max()].index
temp_df = lc_df.drop(temp_index)

temp_index = temp_df[temp_df['tot_coll_amt']==temp_df['tot_coll_amt'].max()].index
temp_df = temp_df.drop(temp_index)

temp_index = temp_df[temp_df['tot_coll_amt']==temp_df['tot_coll_amt'].max()].index
temp_df = temp_df.drop(temp_index)

temp_index = temp_df[temp_df['tot_coll_amt']==temp_df['tot_coll_amt'].max()].index
temp_df = temp_df.drop(temp_index)

temp_index = temp_df[temp_df['tot_coll_amt']==temp_df['tot_coll_amt'].max()].index
temp_df = temp_df.drop(temp_index)

# annual_inc

a = make_pivot_table3('tot_coll_amt',10,temp_df)
a

## 이 데이터는 확인이 필요해보임


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,whole(%),good,bad,bad ratio(%)
0,99.89,915035,176638,16.18
1,0.09,839,136,13.95
2,0.02,140,27,16.17
3,0.00,43,11,20.37
4,0.00,19,5,20.83
5,0.00,6,5,45.45
6,0.00,2,1,33.33
7,0.00,4,0,0.00
8,0.00,1,1,50.00
9,0.00,1,0,0.00


In [61]:
# tot_cur_bal

a = make_pivot_table3('tot_cur_bal',10,temp_df)
a

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,whole(%),good,bad,bad ratio(%)
0,99.50,911020,176393,16.22
1,0.46,4671,407,8.01
2,0.03,321,19,5.59
3,0.01,53,4,7.02
4,0.00,18,1,5.26
5,0.00,4,0,0.00
6,0.00,2,0,0.00
7,0.00,0,0,NaN
8,0.00,0,0,NaN
9,0.00,1,0,0.00


In [64]:
# 위 tot_cur_bal 컬럼에서 너무 큰 이상치가 1개 있어 이값을 제외하고 다시 실행

temp_index = lc_df[lc_df['tot_cur_bal']==lc_df['tot_cur_bal'].max()].index
temp_df = lc_df.drop(temp_index)

# tot_cur_bal

a = make_pivot_table3('tot_cur_bal',10,temp_df)
a

## 이 데이터는 확인이 필요해보임

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,whole(%),good,bad,bad ratio(%)
0,97.87,895000,174642,16.33
1,1.97,19447,2068,9.61
2,0.13,1278,92,6.72
3,0.02,257,14,5.17
4,0.01,69,6,8.00
5,0.00,21,1,4.55
6,0.00,14,1,6.67
7,0.00,4,0,0.00
8,0.00,2,0,0.00
9,0.00,2,0,0.00


In [65]:
# delinq_amnt

a = make_pivot_table3('delinq_amnt',10,temp_df)
a

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,whole(%),good,bad,bad ratio(%)
0,99.97,915868,176763,16.18
1,0.01,99,25,20.16
2,0.01,47,13,21.67
3,0.01,72,21,22.58
4,0.00,3,1,25.00
5,0.00,3,0,0.00
6,0.00,0,1,100.00
7,0.00,1,0,0.00
8,0.00,0,0,NaN
9,0.00,1,0,0.00
